In [44]:
def read_input(filename):
    data = []
    with open(filename) as input:
        for line in input.readlines():
            splitline1 = line.rsplit('Step ', 1)
            first = splitline1[1][0]
            splitline2 = line.rsplit('step ', 1)
            second = splitline2[1][0]
            data.append((first, second))
    return data
            


In [52]:
def find_prerequisites(data):
    prerequisites = {}
    prerequisite_steps = list(zip(*data))[0]
    for step_prerequisite in data:
        prerequisite, step = step_prerequisite
        prerequisites.setdefault(step, []).append(prerequisite)
        
    prerequisites_copy = {**prerequisites}
    
    for step in prerequisite_steps:
        if step not in prerequisites_copy.keys():
            prerequisites[step] = []
    
    return prerequisites

In [53]:
def get_task_order(prerequisite_dict):
    task_order = []
    while True:
        no_prereq = sorted([key for key, val in prerequisite_dict.items() if not val])
        if not no_prereq:
            break
        task = no_prereq[0]
        task_order.append(task)
        del prerequisite_dict[task]
        for step, prereq  in prerequisite_dict.items():
            if task in prereq:
                prereq.remove(task)
    return ''.join(task_order)

In [247]:
def get_task_time(prerequisite_dict, worker_n):
    
    task_time = "_ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    
    workers = {}
    for i in range(worker_n):
        workers[i] = ['_', 0]
    tasks_done = []
    
    total_time = 0
    
    tasks_to_do = []
    tasks_with_time_remaining = []
    while True:
        print('-------')
        print(f'workers - start {workers}')
        print(f'total_time {total_time}')
        print(f'tasks_done {tasks_done}')
        # find tasks that are ready to be done
        
        new_tasks_to_do = [
            key for key, val in prerequisite_dict.items() if not val and key not in tasks_with_time_remaining
        ]
        tasks_to_do += new_tasks_to_do
        tasks_to_do = sorted(tasks_to_do)
        if not tasks_to_do:
            print(f'workers at no tasks to do {workers}')
            tasks_time_remaining = [current_task[1] for worker, current_task in workers.items() if current_task[1]]
            print(max(tasks_time_remaining))
            total_time += max(tasks_time_remaining)
            break
        
        # check for done tasks and assign tasks to workers

        workers_gen = (
            (worker, curr_task_time_left) for worker, curr_task_time_left in workers.items() if curr_task_time_left[1] == 0
        )
        
        copy_tasks_to_do = list(tasks_to_do)
        for task in copy_tasks_to_do:
            try:
                worker, curr_task_time_left = next(workers_gen)
            except StopIteration:
                print('workers at stop {workers}')
                break

#             if curr_task_time_left[0] != '_' and curr_task_time_left[0] not in tasks_done:
#                 tasks_done.append(curr_task_time_left[0])
            
            if task in prerequisite_dict.keys():
                del prerequisite_dict[task]

                workers[worker] = [task, 60 + task_time.index(task)]
            tasks_to_do.remove(task)

            for step, prereq  in prerequisite_dict.items():
                if task in prereq:
                    prereq.remove(task)
                
                
        # find min time from workers and increment
        print(f'workers - mid {workers}')
        tasks_with_time_remaining = [current_task[0] for worker, current_task in workers.items() if current_task[1]]        
        tasks_time_remaining = [current_task[1] for worker, current_task in workers.items() if current_task[1]]
        min_task_time_remaining = min(tasks_time_remaining) if tasks_time_remaining else 0 
        total_time += min_task_time_remaining
        for worker, current_task in workers.items():
            if current_task[1] > 0:
                new_time = current_task[1] - min_task_time_remaining 
                current_task[1] = new_time
                if new_time == 0 and current_task[0] != '_':
                    tasks_done.append(current_task[0])
                    tasks_with_time_remaining.remove(current_task[0])
                
        print(f'workers - end {workers}')
        print(tasks_done)
    return total_time

In [248]:
data = read_input('day7input.txt')

In [249]:
prerequisites = find_prerequisites(data)

In [250]:
# tasks = get_task_order(prerequisites)


In [251]:
prerequisites = find_prerequisites(data)
get_task_time(prerequisites, 5)

-------
workers - start {0: ['_', 0], 1: ['_', 0], 2: ['_', 0], 3: ['_', 0], 4: ['_', 0]}
total_time 0
tasks_done []
workers - mid {0: ['J', 70], 1: ['K', 71], 2: ['X', 84], 3: ['_', 0], 4: ['_', 0]}
workers - end {0: ['J', 0], 1: ['K', 1], 2: ['X', 14], 3: ['_', 0], 4: ['_', 0]}
['J']
-------
workers - start {0: ['J', 0], 1: ['K', 1], 2: ['X', 14], 3: ['_', 0], 4: ['_', 0]}
total_time 70
tasks_done ['J']
workers at stop {workers}
workers - mid {0: ['D', 64], 1: ['K', 1], 2: ['X', 14], 3: ['E', 65], 4: ['P', 76]}
workers - end {0: ['D', 63], 1: ['K', 0], 2: ['X', 13], 3: ['E', 64], 4: ['P', 75]}
['J', 'K']
-------
workers - start {0: ['D', 63], 1: ['K', 0], 2: ['X', 13], 3: ['E', 64], 4: ['P', 75]}
total_time 71
tasks_done ['J', 'K']
workers at stop {workers}
workers - mid {0: ['D', 63], 1: ['F', 66], 2: ['X', 13], 3: ['E', 64], 4: ['P', 75]}
workers - end {0: ['D', 50], 1: ['F', 53], 2: ['X', 0], 3: ['E', 51], 4: ['P', 62]}
['J', 'K', 'X']
-------
workers - start {0: ['D', 50], 1: ['F

534